<a href="https://colab.research.google.com/github/brayvid/soap-research/blob/main/soap_sql_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime

# Get the current date and time
now = datetime.datetime.now()

# Print it
print("Last run:",now)

Last run: 2025-07-12 17:23:36.731327


# DB connection

In [2]:
import os
# from dotenv import load_dotenv
from google.colab import userdata
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2

# Load environment variables from .env file
# load_dotenv()

# Access the variables
# db_host = os.getenv('DB_HOST')
# db_port = os.getenv('DB_PORT', '5432')
# db_name = os.getenv('DB_NAME')
# db_user = os.getenv('DB_USER')
# db_pass = os.getenv('DB_PASS')
db_host = userdata.get('DB_HOST')
db_port = userdata.get('DB_PORT')
db_name = userdata.get('DB_NAME')
db_user = userdata.get('DB_USER')
db_pass = userdata.get('DB_PASS')

db_connection_str = None # Initialize
engine = None # Initialize

if not all([db_host, db_name, db_user, db_pass]):
    print("ERROR: Database credentials not fully loaded from .env or environment.")
    print("Please ensure DB_HOST, DB_NAME, DB_USER, and DB_PASS are in your .env file or environment.")
else:
    print("Database credentials loaded successfully.")
    # Construct the SQLAlchemy connection string
    db_connection_str = f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}'
    try:
        engine = create_engine(db_connection_str)

        # Test connection with a simple query
        # Use a context manager for the connection to ensure it's closed
        with engine.connect() as connection:
            # Wrap the SQL string in text() for direct execution
            result = connection.execute(text("SELECT version();"))
            version_row = result.fetchone() # Fetch one row
            if version_row:
                print(f"\nConnection to PostgreSQL successful! Version: {version_row[0]}")
            else:
                print("\nConnection to PostgreSQL successful, but version query returned no result.")
            # The connection is automatically closed when exiting the 'with' block

    except Exception as e:
        print(f"\nFailed to create SQLAlchemy engine or connect: {e}")
        engine = None # Ensure engine is None if connection failed

Database credentials loaded successfully.

Connection to PostgreSQL successful! Version: PostgreSQL 16.8 (Debian 16.8-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


# All words

In [3]:
if engine:
    # We add COUNT(DISTINCT V.politician_id) to the SELECT list.
    sql_query = """
    SELECT
        W.word_id,
        W.word,
        W.sentiment_score,
        COUNT(V.vote_id) AS total_votes,
        COUNT(DISTINCT V.user_id) AS unique_voters,
        COUNT(DISTINCT V.politician_id) AS politicians_associated -- <-- Added this line
    FROM
        words W
    INNER JOIN
        votes V ON W.word_id = V.word_id
    GROUP BY
        W.word_id, W.word, W.sentiment_score
    ORDER BY
        total_votes DESC;
    """
    try:
        print("--- Query: All words with sentiment, total votes, unique voters, and distinct politicians ---")
        print(sql_query)

        # Execute the query and load the results into a pandas DataFrame
        df_query = pd.read_sql_query(sql_query, engine)

        # Display the resulting DataFrame in the notebook
        display(df_query)

    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
        print("Please check that the 'words' and 'votes' tables exist, along with all specified columns.")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: All words with sentiment, total votes, unique voters, and distinct politicians ---

    SELECT
        W.word_id,
        W.word,
        W.sentiment_score,
        COUNT(V.vote_id) AS total_votes,
        COUNT(DISTINCT V.user_id) AS unique_voters,
        COUNT(DISTINCT V.politician_id) AS politicians_associated -- <-- Added this line
    FROM
        words W
    INNER JOIN
        votes V ON W.word_id = V.word_id
    GROUP BY
        W.word_id, W.word, W.sentiment_score
    ORDER BY
        total_votes DESC;
    


,word_id,word,sentiment_score,total_votes,unique_voters,politicians_associated
0,4,ethical,0.5106,254,19,14
1,11,corrupt,-0.9000,223,13,6
2,78,hateful,-0.4939,185,12,10
3,10,evil,-0.6597,152,11,6
4,146,liar,-0.5106,143,13,3
...,...,...,...,...,...,...
144,217,indecent,-0.7000,1,1,1
145,162,troll,-0.8000,1,1,1
146,150,statesman,0.6000,1,1,1
147,249,buffoon,-0.8000,1,1,1


# All politicians

In [4]:
if engine:
    # This query now joins three tables: politicians -> votes -> words
    # It calculates the average sentiment_score from the words table for each politician.
    sql_query = """
    SELECT
        P.name,
        P.politician_id,
        COUNT(V.vote_id) AS total_votes,
        AVG(W.sentiment_score) AS avg_sentiment
    FROM
        politicians P
    INNER JOIN
        votes V ON P.politician_id = V.politician_id
    INNER JOIN
        words W ON V.word_id = W.word_id               -- <-- Join with the words table
    GROUP BY
        P.politician_id, P.name  -- Group by politician to aggregate their votes
    ORDER BY
        total_votes DESC;        -- Order by the most voted-for politicians
    """
    try:
        print("--- Query: All politicians with total votes and average sentiment score ---")
        print(sql_query)

        # Execute the query and load the results into a pandas DataFrame
        df_query = pd.read_sql_query(sql_query, engine)

        # Display the resulting DataFrame
        display(df_query)

    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
        print("Please check that the 'politicians', 'votes', and 'words' tables exist, along with all specified columns.")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: All politicians with total votes and average sentiment score ---

    SELECT
        P.name,
        P.politician_id,
        COUNT(V.vote_id) AS total_votes,
        AVG(W.sentiment_score) AS avg_sentiment
    FROM
        politicians P
    INNER JOIN
        votes V ON P.politician_id = V.politician_id
    INNER JOIN
        words W ON V.word_id = W.word_id               -- <-- Join with the words table
    GROUP BY
        P.politician_id, P.name  -- Group by politician to aggregate their votes
    ORDER BY
        total_votes DESC;        -- Order by the most voted-for politicians
    


,name,politician_id,total_votes,avg_sentiment
0,Donald Trump,1,1229,-0.518062
1,Bernie Sanders,2,420,0.459672
2,Pete Hegseth,600,213,-0.549903
3,Alexandria Ocasio-Cortez,36,183,0.367182
4,JD Vance,591,178,-0.551169
5,Mitch McConnell,599,176,-0.709974
6,Cory Booker,3,141,0.473599
7,Kristi Noem,624,128,-0.564198
8,Ted Cruz,5,114,-0.458831
9,Elizabeth Warren,597,114,0.111089


# Words for Trump

In [5]:
if engine:
    # --- Set the target politician ID ---
    target_politician_id = 1
    # --- ---

    # We add W.sentiment_score to both the SELECT and GROUP BY clauses.
    sql_query = """
    SELECT
        W.word,
        W.word_id,
        COUNT(V.vote_id) AS votes,
        W.sentiment_score  -- <-- Added sentiment_score here
    FROM
        words W
    INNER JOIN
        votes V ON W.word_id = V.word_id
    WHERE
        V.politician_id = %(pol_id)s
    GROUP BY
        W.word_id, W.word, W.sentiment_score  -- <-- And also added it here
    ORDER BY
        votes DESC;
    """
    try:
        print(f"--- Query: Words (and their sentiment) submitted for Politician ID = {target_politician_id}, ordered by vote count ---")
        print(sql_query)

        # Execute the query with the target ID as a parameter
        df_query = pd.read_sql_query(
            sql_query,
            engine,
            params={'pol_id': target_politician_id}
        )

        # Display the resulting DataFrame
        display(df_query)

    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
        print("Please check that the 'words' and 'votes' tables exist, along with all specified columns.")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: Words (and their sentiment) submitted for Politician ID = 1, ordered by vote count ---

    SELECT
        W.word,
        W.word_id,
        COUNT(V.vote_id) AS votes,
        W.sentiment_score  -- <-- Added sentiment_score here
    FROM
        words W
    INNER JOIN
        votes V ON W.word_id = V.word_id
    WHERE
        V.politician_id = %(pol_id)s
    GROUP BY
        W.word_id, W.word, W.sentiment_score  -- <-- And also added it here
    ORDER BY
        votes DESC;
    


,word,word_id,votes,sentiment_score
0,cruel,35,131,-0.5859
1,corrupt,11,123,-0.9000
2,liar,146,108,-0.5106
3,genius,51,83,0.9500
4,greedy,110,83,-0.3182
5,evil,10,64,-0.6597
6,traitor,142,50,-0.9500
7,asshole,145,44,-1.0000
8,dictator,8,42,-0.9000
9,selfish,14,36,-0.4767


# Words with 1 unique voter

In [6]:
if engine:
    # We add W.sentiment_score to both the SELECT and GROUP BY clauses.
    # The HAVING clause is unaffected and still filters for groups (words) with one unique voter.
    sql_query = """
    SELECT
        W.word,
        W.word_id,
        W.sentiment_score,  -- <-- Added sentiment_score here
        COUNT(V.vote_id) AS votes,
        COUNT(DISTINCT V.politician_id) AS politicians
    FROM
        words W
    INNER JOIN
        votes V ON W.word_id = V.word_id
    GROUP BY
        W.word_id, W.word, W.sentiment_score  -- <-- And also added it here
    HAVING
        COUNT(DISTINCT V.user_id) = 1  -- Filter for words with exactly one unique voter
    ORDER BY
        votes DESC;
    """
    try:
        print("--- Query: Words (and their sentiment) with exactly one unique voter ---")
        print(sql_query)

        # Execute the query and load into a DataFrame
        df_query = pd.read_sql_query(sql_query, engine)

        if not df_query.empty:
            # Display the result if any words match the criteria
            display(df_query)
        else:
            print("\nNo words found that were submitted by only one unique voter.")

    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
        print("Please check that the 'words' and 'votes' tables exist, along with all specified columns.")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: Words (and their sentiment) with exactly one unique voter ---

    SELECT
        W.word,
        W.word_id,
        W.sentiment_score,  -- <-- Added sentiment_score here
        COUNT(V.vote_id) AS votes,
        COUNT(DISTINCT V.politician_id) AS politicians
    FROM
        words W
    INNER JOIN
        votes V ON W.word_id = V.word_id
    GROUP BY
        W.word_id, W.word, W.sentiment_score  -- <-- And also added it here
    HAVING
        COUNT(DISTINCT V.user_id) = 1  -- Filter for words with exactly one unique voter
    ORDER BY
        votes DESC;
    


,word,word_id,sentiment_score,votes,politicians
0,handsome,236,0.4939,30,2
1,eloquent,115,0.6000,28,3
2,loser,213,-0.5267,24,1
3,annoying,224,-0.4019,21,1
4,israeli,245,-0.1000,20,1
5,righteous,242,0.2000,20,1
6,awesome,141,0.6249,18,1
7,reasonable,89,0.6000,18,2
8,useless,138,-0.4215,17,1
9,heartless,120,-0.4939,16,1


# Words with 2 or more unique voters

In [7]:
if engine:
    # We add W.sentiment_score to both the SELECT and GROUP BY clauses.
    # The HAVING and ORDER BY clauses remain unchanged.
    sql_query = """
    SELECT
        W.word,
        W.word_id,
        COUNT(V.vote_id) AS votes,
        W.sentiment_score,  -- <-- Added sentiment_score here
        COUNT(DISTINCT V.politician_id) AS politicians,
        COUNT(DISTINCT V.user_id) AS voters
    FROM
        words W
    INNER JOIN
        votes V ON W.word_id = V.word_id
    GROUP BY
        W.word_id, W.word, W.sentiment_score  -- <-- And also added it here
    HAVING
        COUNT(DISTINCT V.user_id) >= 2  -- Filter for words with at least two unique voters
    ORDER BY
        voters DESC, votes DESC;
    """
    try:
        print("--- Query: All words (and their sentiment) with 2 or more unique voters ---")
        print(sql_query)

        # Execute the query and load into a DataFrame
        df_query = pd.read_sql_query(sql_query, engine)

        if not df_query.empty:
            # Display the result if any words match the criteria
            display(df_query)
        else:
            print("\nNo words found that were submitted by at least two unique voters.")

    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
        print("Please check that the 'words' and 'votes' tables exist, along with all specified columns.")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: All words (and their sentiment) with 2 or more unique voters ---

    SELECT
        W.word,
        W.word_id,
        COUNT(V.vote_id) AS votes,
        W.sentiment_score,  -- <-- Added sentiment_score here
        COUNT(DISTINCT V.politician_id) AS politicians,
        COUNT(DISTINCT V.user_id) AS voters
    FROM
        words W
    INNER JOIN
        votes V ON W.word_id = V.word_id
    GROUP BY
        W.word_id, W.word, W.sentiment_score  -- <-- And also added it here
    HAVING
        COUNT(DISTINCT V.user_id) >= 2  -- Filter for words with at least two unique voters
    ORDER BY
        voters DESC, votes DESC;
    


,word,word_id,votes,sentiment_score,politicians,voters
0,ethical,4,254,0.5106,14,19
1,corrupt,11,223,-0.9000,6,13
2,liar,146,143,-0.5106,3,13
3,cruel,35,131,-0.5859,1,13
4,wise,6,83,0.4767,2,13
...,...,...,...,...,...,...
107,articulate,210,2,0.6000,2,2
108,cop,65,2,-0.2000,1,2
109,sloppy,62,2,-0.6000,1,2
110,hardworking,144,2,0.7500,1,2


# Weekly most used word for Trump

In [8]:
if engine:
    target_politician_id = 1
    # !!! IMPORTANT: Replace 'V.created_at' below with the ACTUAL timestamp column name in your 'votes' table !!!
    actual_timestamp_column = 'V.created_at' # For example, if your column is named 'created_at'

    sql_query = f"""
    WITH WeeklyWordCounts AS (
        -- Step 1: Aggregate vote counts for each word within each week.
        SELECT
            DATE_TRUNC('week', {actual_timestamp_column}) AS week_start,
            W.word_id,
            W.word,
            W.sentiment_score, -- <-- ADDED: Get the sentiment score for the word
            COUNT(V.vote_id) AS word_submissions_in_week,
            COUNT(DISTINCT V.user_id) AS unique_voters_in_week
        FROM
            votes V
        INNER JOIN
            words W ON V.word_id = W.word_id
        WHERE
            V.politician_id = %(pol_id)s
        GROUP BY
            -- Add sentiment_score to the GROUP BY clause as it's not an aggregate
            DATE_TRUNC('week', {actual_timestamp_column}),
            W.word_id,
            W.word,
            W.sentiment_score -- <-- ADDED: Group by it as well
    ),
    RankedWeeklyWords AS (
        -- Step 2: Rank the words within each week based on their submission count.
        SELECT
            week_start,
            word_id,
            word,
            sentiment_score, -- <-- ADDED: Carry the sentiment score through
            word_submissions_in_week,
            unique_voters_in_week,
            ROW_NUMBER() OVER (PARTITION BY week_start
                               ORDER BY word_submissions_in_week DESC, word ASC) as rn
        FROM
            WeeklyWordCounts
    )
    -- Step 3: Select only the top-ranked word (rn = 1) for each week.
    SELECT
        week_start,
        word AS most_used_word,
        sentiment_score, -- <-- ADDED: Display the sentiment score in the final result
        word_submissions_in_week AS votes,
        unique_voters_in_week AS voters
    FROM
        RankedWeeklyWords
    WHERE
        rn = 1
    ORDER BY
        week_start ASC;
    """
    try:
        print(f"--- Query: Most used word (with sentiment) per week for Politician ID = {target_politician_id} ---")

        df_query = pd.read_sql_query(
            sql_query,
            engine,
            params={'pol_id': target_politician_id}
        )
        if not df_query.empty:
            display(df_query)
        else:
            print(f"\nNo weekly word usage data found for Politician ID {target_politician_id}.")
    except Exception as e:
        print(f"AN ERROR OCCURRED:\n{e}")
        if hasattr(e, 'orig') and e.orig:
            print(f"\nOriginal driver error details:\n{e.orig}")
            if hasattr(e.orig, 'pgerror'):
                print(f"PostgreSQL Error Message: {e.orig.pgerror}")
            if hasattr(e.orig, 'diag') and hasattr(e.orig.diag, 'message_detail'):
                 print(f"PostgreSQL Error Detail: {e.orig.diag.message_detail}")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: Most used word (with sentiment) per week for Politician ID = 1 ---


,week_start,most_used_word,sentiment_score,votes,voters
0,2025-04-14 00:00:00+00:00,insane,-0.4019,15,2
1,2025-04-21 00:00:00+00:00,cruel,-0.5859,10,3
2,2025-04-28 00:00:00+00:00,hateful,-0.4939,5,4
3,2025-05-05 00:00:00+00:00,chaotic,-0.4939,1,1
4,2025-05-12 00:00:00+00:00,dangerous,-0.4767,5,1
5,2025-05-19 00:00:00+00:00,borders,-0.1500,4,1
6,2025-05-26 00:00:00+00:00,russian,-0.2000,15,2
7,2025-06-02 00:00:00+00:00,traitor,-0.9500,20,2
8,2025-06-09 00:00:00+00:00,corrupt,-0.9000,6,1
9,2025-06-16 00:00:00+00:00,brazen,-0.7000,11,2


# Count unique users

In [9]:
# This code assumes the previous script part has been run and 'engine' is available.

if engine:
    # --- Code to report the total number of unique submitters ---

    # IMPORTANT: Replace 'user_id' with the actual column name in your 'words' table
    # that identifies the user who submitted the word.
    # Common alternatives: submitter_id, author_id, created_by_user_id
    submitter_column_name = 'user_id' # <<< ---- CHANGE THIS IF NEEDED

    sql_unique_submitters_query = f"""
    SELECT
        COUNT(DISTINCT {submitter_column_name}) AS total_unique_submitters
    FROM
        words;
    """
    try:
        print(f"\n--- Query: Total number of unique submitters from the '{submitter_column_name}' column in 'words' table ---")
        print(sql_unique_submitters_query)

        # Execute the query and get the result into a DataFrame
        df_submitters = pd.read_sql_query(sql_unique_submitters_query, engine)

        # The result will be a DataFrame with one row and one column.
        # We can extract the single value.
        if not df_submitters.empty:
            total_unique_submitters = df_submitters.iloc[0]['total_unique_submitters']
            print(f"\nTotal number of unique submitters: {total_unique_submitters}")
        else:
            print("\nQuery executed, but no result returned (e.g., the 'words' table might be empty or the column doesn't exist).")

        # Optionally, display the DataFrame itself
        # print("\nDataFrame result:")
        # display(df_submitters)

    except Exception as e:
        print(f"Error executing query for unique submitters: {e}")
else:
    print("Database engine not available. Please ensure the connection part of the script was run successfully.")


--- Query: Total number of unique submitters from the 'user_id' column in 'words' table ---

    SELECT
        COUNT(DISTINCT user_id) AS total_unique_submitters
    FROM
        words;
    

Total number of unique submitters: 23


# Lookup sentiment for a list of words

In [10]:
# This code assumes the previous script part has been run and 'engine' is available.

if engine:
    # --- Set the target words to look up in a list ---
    target_words = ['smart', 'principled','evil']
    # --- ---

    # IMPORTANT: Change this if your column name for the sentiment score is different.
    sentiment_column_name = 'sentiment_score'

    # Using the 'IN' operator to find all words in the list.
    # We pass the list of words as a tuple to the query parameters.
    # The database does NOT guarantee the order of results from an IN clause.
    # We will handle the ordering in Pandas after fetching the data.
    sql_query = f"""
    SELECT
        word,
        {sentiment_column_name}
    FROM
        words
    WHERE
        word IN %(word_list)s;
    """
    try:
        print(f"--- Query: Fetch sentiment for a list of words ---")
        print(sql_query)
        print(f"Target words: {target_words}")

        # Execute the query using the list of words.
        # Note: psycopg2 (the driver behind the scenes) requires a tuple for an 'IN' clause,
        # so we convert our list to a tuple.
        df_results = pd.read_sql_query(
            sql_query,
            engine,
            params={'word_list': tuple(target_words)}
        )

        if not df_results.empty:
            print("\n--- Results from the database ---")

            # To enforce the original order of 'target_words', we can use pandas' reindexing.
            # 1. Set the 'word' column as the DataFrame index.
            # 2. Reindex the DataFrame based on the original 'target_words' list.
            # 3. Reset the index to bring 'word' back as a regular column.
            df_ordered = df_results.set_index('word').reindex(target_words).reset_index()

            # Any word from the input list that was not found in the database
            # will now have a NaN (Not a Number) or <NA> value in the sentiment column.

            # Using display() is ideal for showing DataFrames in notebooks like Colab.
            # If not in a notebook, you can use print(df_ordered).
            display(df_ordered)

        else:
            print(f"\nNone of the specified words were found in the 'words' table.")

    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
        print(f"Please check that the 'words' table and the column '{sentiment_column_name}' exist.")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: Fetch sentiment for a list of words ---

    SELECT
        word,
        sentiment_score
    FROM
        words
    WHERE
        word IN %(word_list)s;
    
Target words: ['smart', 'principled', 'evil']

--- Results from the database ---


,word,sentiment_score
0,smart,0.4019
1,principled,0.7500
2,evil,-0.6597


# Count submissions excluding some UIDs

In [11]:
# This code assumes the previous script part has been run and 'engine' is available.

if engine:
    # --- Code to report the total number of submissions, excluding certain users ---

    # IMPORTANT: Replace 'user_id' with the actual column name for the submitter.
    submitter_column_name = 'user_id' # <<< ---- CHANGE THIS IF NEEDED

    # Define the list of user IDs to exclude from the count
    excluded_user_ids = [11, 12, 13]

    # Format the list of IDs for the SQL 'IN' clause.
    # e.g., [12, 13] becomes '(12, 13)'
    # Using a tuple is a good practice as it's immutable and creates the correct parentheses.
    excluded_ids_sql_str = str(tuple(excluded_user_ids))

    # SQL query to count all rows in the 'words' table where the submitter is NOT in the excluded list.
    # COUNT(*) is used to count all rows that match the WHERE clause.
    sql_submissions_query = f"""
    SELECT
        COUNT(*) AS total_submissions_excluding_users
    FROM
        votes
    WHERE
        {submitter_column_name} NOT IN {excluded_ids_sql_str};
    """
    try:
        print(f"\n--- Query: Total submissions from users other than IDs {excluded_user_ids} ---")
        print(sql_submissions_query)

        # Execute the query and get the result into a DataFrame
        df_submissions = pd.read_sql_query(sql_submissions_query, engine)

        # The result will be a DataFrame with one row and one column.
        # We can extract the single value.
        if not df_submissions.empty:
            # The column name 'total_submissions_excluding_users' must match the alias in the SQL query
            total_submissions = df_submissions.iloc[0]['total_submissions_excluding_users']
            print(f"\nSubmissions: {total_submissions}")
        else:
            # A COUNT query always returns a row, even if the count is 0.
            # This 'else' block would typically only be hit in case of a strange database error.
            print("\nQuery executed, but no result was returned.")

    except Exception as e:
        print(f"Error executing query for total submissions: {e}")
else:
    print("Database engine not available. Please ensure the connection part of the script was run successfully.")


--- Query: Total submissions from users other than IDs [11, 12, 13] ---

    SELECT
        COUNT(*) AS total_submissions_excluding_users
    FROM
        votes
    WHERE
        user_id NOT IN (11, 12, 13);
    

Submissions: 1517


# Most active UIDs

In [12]:
if engine:
    sql_query = """
    SELECT
        V.user_id,
        COUNT(V.vote_id) AS total_votes_cast
    FROM
        votes V
    GROUP BY
        V.user_id
    ORDER BY
        total_votes_cast DESC
    LIMIT 20;
    """
    try:
        print("--- Query: Top 10 most active voters by total votes cast ---")
        print(sql_query)
        df_query = pd.read_sql_query(sql_query, engine)
        if not df_query.empty:
            display(df_query)
        else:
            print("\nNo voting data found in the 'votes' table.")
    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: Top 10 most active voters by total votes cast ---

    SELECT
        V.user_id,
        COUNT(V.vote_id) AS total_votes_cast
    FROM
        votes V
    GROUP BY
        V.user_id
    ORDER BY
        total_votes_cast DESC
    LIMIT 20;
    


,user_id,total_votes_cast
0,13,2560
1,75,466
2,61,394
3,11,261
4,1,122
5,54,55
6,82,50
7,81,37
8,53,35
9,70,32


# User sentiment profiles

In [13]:
if engine:
    sql_query = """
    SELECT
        V.user_id,
        COUNT(V.vote_id) AS total_votes,
        AVG(W.sentiment_score) AS average_sentiment
    FROM
        votes V
    INNER JOIN
        words W ON V.word_id = W.word_id
    GROUP BY
        V.user_id
    HAVING
        COUNT(V.vote_id) > 5 -- Optional: Only show users with more than 10 votes.
    ORDER BY
        average_sentiment DESC; -- Shows most negative first. Use DESC for most positive.
    """
    try:
        print("--- Query: User sentiment profiles (for users with >10 votes) ---")
        print(sql_query)
        df_query = pd.read_sql_query(sql_query, engine)
        if not df_query.empty:
            display(df_query)
        else:
            print("\nNo users found matching the criteria (e.g., none with >10 votes).")
    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: User sentiment profiles (for users with >10 votes) ---

    SELECT
        V.user_id,
        COUNT(V.vote_id) AS total_votes,
        AVG(W.sentiment_score) AS average_sentiment
    FROM
        votes V
    INNER JOIN
        words W ON V.word_id = W.word_id
    GROUP BY
        V.user_id
    HAVING
        COUNT(V.vote_id) > 5 -- Optional: Only show users with more than 10 votes.
    ORDER BY
        average_sentiment DESC; -- Shows most negative first. Use DESC for most positive.
    


,user_id,total_votes,average_sentiment
0,57,10,0.805610
1,77,10,0.803940
2,73,10,0.600000
3,58,6,0.553583
4,72,10,0.523860
5,23,12,0.190033
6,16,14,0.162307
7,75,466,0.066727
8,1,122,0.064023
9,76,18,0.009939


# Weekly sentiment trajectory for Trump

In [14]:
if engine:
    # --- Set the target politician ID and timestamp column ---
    target_politician_id = 1
    # !!! IMPORTANT: Replace 'V.created_at' below with the ACTUAL timestamp column name in your 'votes' table !!!
    actual_timestamp_column = 'V.created_at'

    sql_query = f"""
    SELECT
        DATE_TRUNC('week', {actual_timestamp_column}) AS week,
        AVG(W.sentiment_score) AS avg_sentiment,
        COUNT(V.vote_id) AS weekly_votes
    FROM
        votes V
    INNER JOIN
        words W ON V.word_id = W.word_id
    WHERE
        V.politician_id = %(pol_id)s
    GROUP BY
        week
    ORDER BY
        week ASC;
    """
    try:
        print(f"--- Query: Weekly sentiment trajectory for Politician ID = {target_politician_id} ---")
        # print(sql_query) # Uncomment for debugging

        df_query = pd.read_sql_query(
            sql_query,
            engine,
            params={'pol_id': target_politician_id}
        )
        if not df_query.empty:
            display(df_query)
        else:
            print(f"\nNo weekly sentiment data found for Politician ID {target_politician_id}.")
    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: Weekly sentiment trajectory for Politician ID = 1 ---


,week,avg_sentiment,weekly_votes
0,2025-04-14 00:00:00+00:00,-0.611719,53
1,2025-04-21 00:00:00+00:00,-0.310193,28
2,2025-04-28 00:00:00+00:00,-0.503450,24
3,2025-05-05 00:00:00+00:00,-0.509450,4
4,2025-05-12 00:00:00+00:00,-0.561223,22
5,2025-05-19 00:00:00+00:00,-0.332454,26
6,2025-05-26 00:00:00+00:00,-0.510120,127
7,2025-06-02 00:00:00+00:00,-0.644005,126
8,2025-06-09 00:00:00+00:00,-0.734650,42
9,2025-06-16 00:00:00+00:00,-0.659516,76


# Most polarizing politicians

In [15]:
if engine:
    sql_query = """
    SELECT
        P.name,
        COUNT(V.vote_id) AS total_votes,
        AVG(W.sentiment_score) AS mean_sentiment,
        STDDEV(W.sentiment_score) AS sentiment_polarity_score
    FROM
        politicians P
    INNER JOIN
        votes V ON P.politician_id = V.politician_id
    INNER JOIN
        words W ON V.word_id = W.word_id
    GROUP BY
        P.politician_id, P.name
    HAVING
        COUNT(V.vote_id) > 20 -- Optional: Only analyze politicians with a meaningful number of votes
    ORDER BY
        sentiment_polarity_score DESC
    LIMIT 20;
    """
    try:
        print("--- Query: Most polarizing politicians (by standard deviation of sentiment) ---")
        print(sql_query)
        df_query = pd.read_sql_query(sql_query, engine)
        if not df_query.empty:
            display(df_query)
        else:
            print("\nNo politicians found matching the criteria (e.g., none with >20 votes).")
    except Exception as e:
        print(f"\nAn error occurred while executing the query: {e}")
else:
    print("Database engine not available. Please run the connection cell first.")

--- Query: Most polarizing politicians (by standard deviation of sentiment) ---

    SELECT
        P.name,
        COUNT(V.vote_id) AS total_votes,
        AVG(W.sentiment_score) AS mean_sentiment,
        STDDEV(W.sentiment_score) AS sentiment_polarity_score
    FROM
        politicians P
    INNER JOIN
        votes V ON P.politician_id = V.politician_id
    INNER JOIN
        words W ON V.word_id = W.word_id
    GROUP BY
        P.politician_id, P.name
    HAVING
        COUNT(V.vote_id) > 20 -- Optional: Only analyze politicians with a meaningful number of votes
    ORDER BY
        sentiment_polarity_score DESC
    LIMIT 20;
    


,name,total_votes,mean_sentiment,sentiment_polarity_score
0,Mike Johnson,93,-0.041935,0.731522
1,Clarence Thomas,93,-0.415009,0.575432
2,Elizabeth Warren,114,0.111089,0.479983
3,Donald Trump,1229,-0.518062,0.478573
4,Tom Cotton,62,-0.184655,0.473369
5,Alexandria Ocasio-Cortez,183,0.367182,0.462735
6,Zohran Mamdani,103,0.163533,0.438438
7,Bernie Sanders,420,0.459672,0.416289
8,Marjorie Taylor Greene,113,-0.482867,0.325332
9,Benjamin Netanyahu,62,-0.258047,0.319243
